<a href="https://colab.research.google.com/github/rudysemola/CL-experiments/blob/main/CLaaS-DEMO-v1-Exp/CLaaS_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLaaS v1

In [ ]:
!pip install git+https://github.com/ContinualAI/avalanche.git

# SplitMNIST 

In [5]:
import avalanche
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import accuracy_metrics, \
    timing_metrics, cpu_usage_metrics, disk_usage_metrics, gpu_usage_metrics, ram_usage_metrics
from avalanche.training.strategies import Replay
from avalanche.training.strategies import Naive, Cumulative

import torch
from torch.optim import SGD
from torch.nn import CrossEntropyLoss

In [12]:
# scenario
scenario = SplitMNIST(n_experiences=5)

# MODEL CREATION
model = SimpleMLP(num_classes=scenario.n_classes)

#logger
monitoring_logger = InteractiveLogger()

#gpu
device = torch.device(f"cuda:{0}" if torch.cuda.is_available() else "cpu")
#Eval plugin
eval_plugin = EvaluationPlugin(
            accuracy_metrics(epoch=True, experience=True, stream=True),
            timing_metrics(epoch=True, experience=True),
            cpu_usage_metrics(experience=True),
            disk_usage_metrics(epoch=True, experience=True, stream=True),
            ram_usage_metrics(epoch=True, experience=True, stream=True),
            gpu_usage_metrics(gpu_id=device.index,epoch=True, experience=True, stream=True),
            loggers=[monitoring_logger]
        ) 

/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:85: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  "No benchmark provided to the evaluation plugin. "


## Naive

In [18]:
# CREATE THE STRATEGY INSTANCE (NAIVE)
naive_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results_naive = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res_naive = naive_strategy.train(experience, num_workers=4)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # eval also returns a dictionary which contains all the metric values
    results_naive.append(naive_strategy.eval(scenario.test_stream, num_workers=4))

Starting experiment...
Start of experience:  0
Current Classes:  [3, 4]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 24/24 [00:05<00:00,  4.43it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 55532.3242
	MaxGPU0Usage_Epoch/train_phase/train_stream/Task000 = 0
	MaxRAMUsage_Epoch/train_phase/train_stream/Task000 = 3303.4922
	Time_Epoch/train_phase/train_stream/Task000 = 5.2530
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7837
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 20/20 [00:02<00:00,  8.12it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 31.1500
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 55532.3242
	MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Exp000 = 0
	MaxRAMUsage_Experience/eval_phase/test_stream/Task000/Exp000 = 3303.4961
	Time_Exp/eval_phase/test_stream/Task000/Exp000 = 2.2151
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9864
-- Starting eval on experience 1 (Task 0) from test stream --


...

0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 19/19 [00:01<00:00, 15.96it/s]
> Eval on experience 3 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp003 = 53.4714
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp003 = 55532.3242
	MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Exp003 = 0
	MaxRAMUsage_Experience/eval_phase/test_stream/Task000/Exp003 = 3303.5117
	Time_Exp/eval_phase/test_stream/Task000/Exp003 = 1.1789
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003 = 0.8691
-- Starting eval on experience 4 (Task 0) from test stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 20/20 [00:01<00:00, 15.56it/s]
> Eval on experience 4 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp004 = 50.1082
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp004 = 55532.3242
	MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Exp004 = 0
	MaxRAMUsage_Experience/eval_phase/test_stream/Task000/Exp004 = 3303.5117
	Time_Exp/eval_phase/test_stream/Task000/Exp004 = 1.1575
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp004 = 0.9718
-- >> End of eval phase << --
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 55532.3242
	MaxGPU0Usage_Stream/eval_phase/test_stream/Task000 = 0
	MaxRAMUsage_Stream/eval_phase/test_stream/Task000 = 3303.5117
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.6775


In [19]:
results_naive

[{'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000': 31.150012594375376,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001': 33.2660874295872,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp002': 28.26802677832152,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp003': 30.617605629535742,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp004': 31.848632214796762,
  'DiskUsage_Epoch/train_phase/train_stream/Task000': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp002': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp003': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp004': 55532.32421875,
  'DiskUsage_Stream/eval_phase/test_stream/Task000': 55532.32421875,
  'MaxGPU0Usage_Epoch/train_phase/train_stream/Task000': 0,
  'MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/E

## Cumulative

In [20]:
# CREATE THE STRATEGY INSTANCE (NAIVE)
cum_strategy= Cumulative(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results_cum = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cum_strategy.train(experience, num_workers=4)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # eval also returns a dictionary which contains all the metric values
    results_cum.append(cum_strategy.eval(scenario.test_stream, num_workers=4))

Starting experiment...
Start of experience:  0
Current Classes:  [3, 4]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 24/24 [00:04<00:00,  5.27it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 55532.3242
	MaxGPU0Usage_Epoch/train_phase/train_stream/Task000 = 0
	MaxRAMUsage_Epoch/train_phase/train_stream/Task000 = 3303.4961
	Time_Epoch/train_phase/train_stream/Task000 = 4.4854
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7917
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0it [00:00, ?it/s]

...

100%|██████████| 20/20 [00:01<00:00, 16.57it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp002 = 49.7340
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp002 = 55532.3242
	MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Exp002 = 0
	MaxRAMUsage_Experience/eval_phase/test_stream/Task000/Exp002 = 3306.0391
	Time_Exp/eval_phase/test_stream/Task000/Exp002 = 1.1266
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.8785
-- Starting eval on experience 3 (Task 0) from test stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 19/19 [00:01<00:00, 15.70it/s]
> Eval on experience 3 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp003 = 52.3826
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp003 = 55532.3242
	MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Exp003 = 0
	MaxRAMUsage_Experience/eval_phase/test_stream/Task000/Exp003 = 3306.0469
	Time_Exp/eval_phase/test_stream/Task000/Exp003 = 1.1273
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003 = 0.9017
-- Starting eval on experience 4 (Task 0) from test stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 20/20 [00:01<00:00, 15.55it/s]
> Eval on experience 4 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp004 = 49.4767
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp004 = 55532.3242
	MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Exp004 = 0
	MaxRAMUsage_Experience/eval_phase/test_stream/Task000/Exp004 = 3306.0508
	Time_Exp/eval_phase/test_stream/Task000/Exp004 = 1.1324
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp004 = 0.9079
-- >> End of eval phase << --
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 55532.3242
	MaxGPU0Usage_Stream/eval_phase/test_stream/Task000 = 0
	MaxRAMUsage_Stream/eval_phase/test_stream/Task000 = 3306.0625
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9064


In [21]:
results_cum

[{'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000': 53.52102047416401,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001': 55.241688127949566,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp002': 50.07811664829793,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp003': 56.94429676841072,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp004': 50.93299166410222,
  'DiskUsage_Epoch/train_phase/train_stream/Task000': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp002': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp003': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp004': 55532.32421875,
  'DiskUsage_Stream/eval_phase/test_stream/Task000': 55532.32421875,
  'MaxGPU0Usage_Epoch/train_phase/train_stream/Task000': 0,
  'MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Ex

## CL Strategy (Random Replay)

In [22]:
# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy_replay = Replay(
            model, SGD(model.parameters(), lr=0.001, momentum=0.9),
            CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
            evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results_cl = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_replay.train(experience, num_workers=4)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # eval also returns a dictionary which contains all the metric values
    results_cl.append(cl_strategy_replay.eval(scenario.test_stream, num_workers=4))

Starting experiment...
Start of experience:  0
Current Classes:  [3, 4]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 24/24 [00:04<00:00,  5.21it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 55532.3242
	MaxGPU0Usage_Epoch/train_phase/train_stream/Task000 = 0
	MaxRAMUsage_Epoch/train_phase/train_stream/Task000 = 3306.0352
	Time_Epoch/train_phase/train_stream/Task000 = 4.5296
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9305
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 20/20 [00:01<00:00, 15.89it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 51.1633
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 55532.3242
	MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Exp000 = 0
	MaxRAMUsage_Experience/eval_phase/test_stream/Task000/Exp000 = 3306.8008
	Time_Exp/eval_phase/test_stream/Task000/Exp000 = 1.1930
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9819
-- Starting eval on experience 1 (Task 0) from test stream --


...

100%|██████████| 20/20 [00:01<00:00, 15.08it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp002 = 47.6754
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp002 = 55532.3242
	MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Exp002 = 0
	MaxRAMUsage_Experience/eval_phase/test_stream/Task000/Exp002 = 3308.9492
	Time_Exp/eval_phase/test_stream/Task000/Exp002 = 1.2583
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.7897
-- Starting eval on experience 3 (Task 0) from test stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 19/19 [00:01<00:00, 13.55it/s]
> Eval on experience 3 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp003 = 50.6066
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp003 = 55532.3242
	MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Exp003 = 0
	MaxRAMUsage_Experience/eval_phase/test_stream/Task000/Exp003 = 3308.9531
	Time_Exp/eval_phase/test_stream/Task000/Exp003 = 1.3042
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003 = 0.8980
-- Starting eval on experience 4 (Task 0) from test stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 20/20 [00:01<00:00, 14.38it/s]
> Eval on experience 4 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp004 = 46.6933
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp004 = 55532.3242
	MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Exp004 = 0
	MaxRAMUsage_Experience/eval_phase/test_stream/Task000/Exp004 = 3308.9766
	Time_Exp/eval_phase/test_stream/Task000/Exp004 = 1.2853
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp004 = 0.9597
-- >> End of eval phase << --
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 55532.3242
	MaxGPU0Usage_Stream/eval_phase/test_stream/Task000 = 0
	MaxRAMUsage_Stream/eval_phase/test_stream/Task000 = 3308.9766
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.8820


In [23]:
results_cl

[{'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000': 51.16330504936534,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001': 52.91518854964126,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp002': 52.76507541971666,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp003': 50.653716400855345,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp004': 51.311224261214846,
  'DiskUsage_Epoch/train_phase/train_stream/Task000': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp002': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp003': 55532.32421875,
  'DiskUsage_Exp/eval_phase/test_stream/Task000/Exp004': 55532.32421875,
  'DiskUsage_Stream/eval_phase/test_stream/Task000': 55532.32421875,
  'MaxGPU0Usage_Epoch/train_phase/train_stream/Task000': 0,
  'MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/E

## Results and Plot

- **Performance** (Stream Accuracy)
- **Efficiency** ( timing_metrics, cpu_usage_metrics, disk_usage_metrics, gpu_usage_metrics, ram_usage_metrics) -> CL/N% CL/Cum%

PLOT 
- Acc | Eff metrics | in experiences

# CORe50

## Naive

## Cumulative

## CL strategy (Random Replay)

## Results and Plot

- **Performance** (Stream Accuracy)
- **Efficiency** ( timing_metrics, cpu_usage_metrics, disk_usage_metrics, gpu_usage_metrics, ram_usage_metrics)

What Else?